#Reading and Writing Data

##A) mat files (scipy.io)

In [110]:
import numpy as np
from scipy.io import loadmat as loadmat #this is the scipy module that loads .mat files
from scipy.io import savemat as savemat #this is the scipy module that saves .mat files

In [155]:
uri = 'http://www.karensmith.squarespace.com/storage/python_test.mat' #can't seem to get loadmat to work with a uri

In [156]:
matfile = loadmat('python_test.mat')  # load .mat file

In [122]:
array1 = matfile['array1']
array2 = matfile['array2']

In [123]:
array1.shape

(20, 2)

In [124]:
type(array1)

numpy.ndarray

In [125]:
savemat('python_test_save.mat',{'array1':array1}) #save numpy array to .mat file

##B) csv, txt, xls

In [126]:
import pandas as pd #great for reading .csv and .txt files

In [127]:
uri1 = 'http://www.ldeo.columbia.edu/~rpa/usgs_earthquakes_2014.csv' #example from Ryan's worskshop

In [128]:
d1 = pd.read_csv(uri1,index_col='time') #many argument options (see pandas website for all the details)

In [53]:
type(d1)

pandas.core.frame.DataFrame

In [60]:
d1.head()

,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type
time,,,,,,,,,,,,,,
2014-01-31 23:53:37.000,60.252000,-152.7081,90.20,1.10,ml,NaN,NaN,NaN,0.2900,ak,ak11155107,2014-02-05T19:34:41.515Z,"26km S of Redoubt Volcano, Alaska",earthquake
2014-01-31 23:48:35.452,37.070300,-115.1309,0.00,1.33,ml,4,171.43,0.34200,0.0247,nn,nn00436847,2014-02-01T01:35:09.000Z,"32km S of Alamo, Nevada",earthquake
2014-01-31 23:47:24.000,64.671700,-149.2528,7.10,1.30,ml,NaN,NaN,NaN,1.0000,ak,ak11151142,2014-02-01T00:03:53.010Z,"12km NNW of North Nenana, Alaska",earthquake
2014-01-31 23:30:54.000,63.188700,-148.9575,96.50,0.80,ml,NaN,NaN,NaN,1.0700,ak,ak11151135,2014-01-31T23:41:25.007Z,"22km S of Cantwell, Alaska",earthquake
2014-01-31 23:30:52.210,32.616833,-115.6925,10.59,1.34,ml,6,285.00,0.04321,0.2000,ci,ci37171541,2014-02-01T00:13:20.107Z,"10km WNW of Progreso, Mexico",earthquake


In [97]:
uri2 = 'http://karensmith.squarespace.com/storage/python_test.csv'

In [98]:
d2 = pd.read_csv(uri2) #default case (no arguments)

In [99]:
d2.head()

,date,A,B,C
0,20151001,1,a,2.5
1,20151002,2,b,5.0
2,20151003,3,c,7.5


In [23]:
d2 = pd.read_csv(uri2,index_col=1) #can pass arguments to specify column order

In [24]:
print(d2)

       date  B    C
A                  
1  20151001  a  2.5
2  20151002  b  5.0
3  20151003  c  7.5


In [100]:
uri3 = 'http://karensmith.squarespace.com/storage/python_test.xls'

In [101]:
d3 = pd.read_excel(uri3) #pandas can also be used to read .xls files

In [102]:
d3.head()

,date,A,B,C
0,20151001,1,a,2.5
1,20151002,2,b,5.0
2,20151003,3,c,7.5


In [30]:
d1.to_csv('earthquakes_test.csv') #writing our d1 DataFrame object to a .csv file

In [31]:
d2.to_excel('new_python_test.xls', sheet_name='Sheet1') #writing our d2 DataFrame object to a .xls file

##C) netcdf4

In [65]:
from netCDF4 import Dataset

In [62]:
uri = 'http://iridl.ldeo.columbia.edu/SOURCES/.NOAA/.NCDC/.ERSST/.version4/anom/.sst/T/(days%20since%201960-01-01)/streamgridunitconvert/dods'

#use 'Dataset' to read file as netcdf4
nc = Dataset(uri)

In [63]:
nc

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format UNDEFINED):
    Conventions: IRIDL
    dimensions(sizes): T(1941), X(180), Y(89), zlev(1)
    variables(dimensions): float32 zlev(zlev), float32 X(X), float32 Y(Y), float32 T(T), int16 sst(T,zlev,Y,X)
    groups: 

In [64]:
SST = nc.variables['sst'][:,0] #this is the same as ['sst'[:,0,:,:]] -> gets rid of a degenerate dimension (same as squeeze in matlab)
Lat = nc.variables['Y'][:]
Lon = nc.variables['X'][:]

In [83]:
#write a new netcdf file
new_nc = Dataset('python_test.nc', 'w', format='NETCDF3_CLASSIC')
new_nc.description = 'Example data'

# define dimensions
new_nc.createDimension('time', None) #record dimension
new_nc.createDimension('lat', 72)
new_nc.createDimension('lon', 144)

# define variables
times = new_nc.createVariable('time', 'f8', ('time',))
latitudes = new_nc.createVariable('latitude', 'f4', ('lat',))
longitudes = new_nc.createVariable('longitude', 'f4', ('lon',))
tmp = new_nc.createVariable('tmp', 'f4', ('time', 'lat', 'lon',))

# allocate data
lats =  np.arange(-90, 90, 2.5)
lons =  np.arange(-180, 180, 2.5)
latitudes[:] = lats
longitudes[:] = lons
for i in range(5):
    tmp[i,:,:] = np.random.uniform(size=(len(lats), len(lons))) #default uniform distribution between 0 and 1

new_nc.close()

##D) netcdf3

In [93]:
from scipy.io import netcdf #scipy.io can only read/write netcdf3

#read the file we just created above using netcdf4
f = netcdf.netcdf_file('python_test.nc', 'r')
f


In [95]:
print(f.description)
lat = f.variables['latitude']
print(lat.shape)
print(lat[:])

b'Example data'
(72,)
[-90.  -87.5 -85.  -82.5 -80.  -77.5 -75.  -72.5 -70.  -67.5 -65.  -62.5
 -60.  -57.5 -55.  -52.5 -50.  -47.5 -45.  -42.5 -40.  -37.5 -35.  -32.5
 -30.  -27.5 -25.  -22.5 -20.  -17.5 -15.  -12.5 -10.   -7.5  -5.   -2.5
   0.    2.5   5.    7.5  10.   12.5  15.   17.5  20.   22.5  25.   27.5
  30.   32.5  35.   37.5  40.   42.5  45.   47.5  50.   52.5  55.   57.5
  60.   62.5  65.   67.5  70.   72.5  75.   77.5  80.   82.5  85.   87.5]


In [96]:
f.close()
#data has to be copied to main memory if we want to process data after we close the netcdf file (see message below)

/Users/karensmith/anaconda/lib/python3.4/site-packages/scipy/io/netcdf.py:287: RuntimeWarning: Cannot close a netcdf_file opened with mmap=True, when netcdf_variables or arrays referring to its data still exist. All data arrays obtained from such files refer directly to data on disk, and must be copied before the file can be cleanly closed. (See netcdf_file docstring for more information on mmap.)
  ), category=RuntimeWarning)


##E) hdf5

In [88]:
import h5py